# Household dust data

A problem in linear regression, using the `homes.Rdata` dataset, extracted from the [Homes_USA](http://figshare.com/articles/1000homes/1270900) repository of data analyzed in the paper [Barberan et al. (2015), _"The ecology of microscopic life in household dust",_](https://royalsocietypublishing.org/doi/full/10.1098/rspb.2015.1139) Proceedings of the Royal Society B, Biological Sciences Volume 282, Issue 1814.
	
The data are dust samples from the ledges above doorways from $n=1,059$ homes (after removing samples with missing data) in the continental US. Bioinformatics processing detects the presence or absence of $763$ species (technically operational taxonomic units) of fungi. The response is the log of the number of fungi species present in the sample, which is a measure of species richness. The objective is to determine which factors influence a home's species richness. For each home, eight covariates are included in this example: 

01. `long`:     Longitude,
02. `lat`:      Latitude,
03. `temp`:     Temperature,
04. `precip`:   Precipitation,
05. `NPP`:      Net Primary Productivity (NPP) (See the  [Appendix](https://www.pnas.org/content/pnas/suppl/2015/04/15/1420815112.DCSupplemental/pnas.1420815112.sapp.pdf)),
06. `elev`:     Elevation,
07. `house`:    Single-family home (binary indicator),
08. `bedrooms`: Number of bedrooms.

In [ ]:
# Load data (there is a warning about NAs. It's OK, later on we remove them)
load("homes.RData")

lat      <- homes[,4]
long     <- homes[,5]
temp     <- homes[,6]
precip   <- homes[,7]
NPP      <- homes[,8]
elev     <- homes[,9]
house    <- ifelse(homes[,10]=="One-family house detached from any other house",1,0) # Convert to an indicator
bedrooms <- as.numeric(homes[,11])

In [ ]:
city     <- homes[,2]
state    <- homes[,3]

OTU      <- as.matrix(OTU)
nspecies <- rowSums(OTU>0)
y        <- log(nspecies)
X        <- cbind(long,lat,temp,precip,NPP,elev,house,bedrooms)
names    <- c("Longitude","Latitude","Temperature","Precipitation","NPP","Elevation","Single-family home",
                "Number of bedrooms")

In [ ]:
# Remove observations with missing data

junk     <- is.na(rowSums(X))
y        <- y[!junk]
X        <- X[!junk,]
city     <- city[!junk]
state    <- state[!junk]

In [ ]:
# Standardize the covariates

X<- as.matrix(scale(X))
X0<-X
X0.centers<-attr(X0,"scaled:center")
X0.scales<-attr(X0,"scaled:scale")
attr(X,"scaled:center")<-NULL
attr(X,"scaled:scale")<-NULL
str(X0.centers)
str(X0.scales)

In [ ]:
str(X)
str(y)

## Plot the sample locations

In [ ]:
#install.packages("maps",dependencies=TRUE,repos= "https://cloud.r-project.org")
require(maps)

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
map("state")
points(homes[,5],homes[,4],pch=19,cex=.5)
title("Sample locations")

# OLS

In [ ]:
# Overwrite the original "homes" data.frame
# For lm() we need a unique data.frame
# For glmnet() X and y
homes<-data.frame(cbind(y,X))
str(homes)

In [ ]:
homes.ols.01<-lm(y~.,data=homes)
summary(homes.ols.01)

In [ ]:
anova(homes.ols.01)

In [ ]:
sigma(homes.ols.01)

In [ ]:
#install.packages("car",dependencies=TRUE,repos= "https://cloud.r-project.org")
require(car)

In [ ]:
round(vif(homes.ols.01),1)

In [ ]:
homes.ols.02<-lm(y~(long+lat+temp+precip+NPP+elev+house+bedrooms)^2,data=homes)
summary(homes.ols.02)

In [ ]:
homes.ols.03<-lm(y~(long+lat+temp+precip+NPP+elev+house+bedrooms)^2
                 +I(long^2)+I(lat^2)+I(temp^2)+I(precip^2)+I(NPP^2)+I(elev^2)
                 +I(bedrooms^2),data=homes)
summary(homes.ols.03)

In [ ]:
anova(homes.ols.03)

In [ ]:
round(vif(homes.ols.03),1)

### Stepwise selection of predictors

In [ ]:
homes.ols.04<-lm(formula = y ~ long + lat + temp + precip + NPP + elev + house + bedrooms 
                 + I(long^2) + I(precip^2) + I(NPP^2) + I(bedrooms^2) 
                 + long:NPP + lat:temp + temp:precip + temp:NPP + temp:house
                 + temp:bedrooms + precip:NPP + NPP:bedrooms, data = homes)
summary(homes.ols.04)

In [ ]:
round(vif(homes.ols.04),1)